## Welcome to this tutorial on how to train your own nanotextural demixing model using the `NanTex` library! 
In this tutorial, we will focus on the second part of the process: constructing and training the network using the `Hypnos` module of the `NanTex` library.

### Requirements
- Synthetic overlay data generated with the `Tekhne` module of the `NanTex` package (see `tekhne_tutorial.ipynb`)
- Tested installation of PyTorch, ideally with GPU support (see `installation_guide.md`)
- Tested installation of TensorBoard (see `installation_guide.md`)
- `NanTex` package installed (see `installation_guide.md`)
- Basic knowledge of Python and Jupyter Notebooks

If you haven't completed the first part of the process, please refer to the `tekhne_tutorial.ipynb` notebook to generate synthetic overlay data. This data will be used to train the demixing model in this tutorial.

## Part 0: Dependencies

In [ ]:
## Dependencies
import os
import json
import torch
from pprint import pprint

# Model (Hypnos)
from nantex.deep_learning.dl_building_blocks import train
from nantex.deep_learning.dl_model_assembly import (
    assembled_model,
    hyperparameters,
)

# Dataloaders (Harmonia)
from nantex.batching import Harmonia

## Part I: Configure and Instantiate Harmonia

In [ ]:
## Generate Config

# define and create config directory
config_dir = '../configs/'
os.makedirs(config_dir, exist_ok=True)

# generate boilerplate config file
Harmonia.generate_boilerplate_config_file(config_dir)

# show the generated config file
with open(os.path.join(config_dir, 'harmonia_config.json'), 'r') as f:
    print(f.read())

**Heads-up** \\\ Some of our modules contain convenience functions that interface with the Windows file system. If you are using a UNIX based system (Linux, MacOS), you will need to provide the directory paths manually in the configuration files. Please refer to the docstrings of the respective functions for more information.

## UNIX

In [ ]:
## define train and validation directories
raw_source:str = '../path/to/directory/containing/training/olverays/'
val_source:str = '../path/to/directory/containing/validation/olverays/'

# write to config file
with open(os.path.join(config_dir, 'harmonia_config.json'), 'r') as f:
    config = json.load(f)

# update config with new paths
config['raw_source'] = raw_source
config['val_source'] = val_source

# write updated config back to file
with open(os.path.join(config_dir, 'harmonia_config.json'), 'w') as f:
    json.dump(config, f, indent=4)

In [ ]:
## Instantiate
BatchProvider:Harmonia
BatchProvider = Harmonia.from_config(config_file_path='../configs/harmonia_config.json',
                                     datatype='npy',
                                     DEBUG=True)

# the configuration can also be passed as a dictionary
with open('../configs/harmonia_config.json', 'r') as f:
    config = json.load(f)
    
BatchProvider = Harmonia(config=config,
                         datatype='npy',
                         DEBUG=True)

## Windows

In [ ]:
## Instantiate
BatchProvider:Harmonia
BatchProvider = Harmonia.from_config(config_file_path='../configs/harmonia_config.json',
                                     datatype='npy',
                                     DEBUG=True)

### Checkpoint I: Instantiate the `Harmonia` class

In [ ]:
## Let's check the configuration
BatchProvider.pprint_config()

## Part II: Configure CUDA and Hyperparameters

In [ ]:
## Check if CUDA is available

# set backend flags
torch.backends.cudnn.enabled = True  # use CUDNN Cuda Deep Neural Network library
torch.backends.cudnn.benchmark = True  # enable benchmarking for optimized performance -> benchmark multiple convolution algorithms and choose the fastest

print("Checking CUDA availability...")
print("  version:", torch.__version__)
print("  CUDA available:", torch.cuda.is_available())
print("  cuDNN available:", torch.backends.cudnn.is_available())
print()
print("Checking on GPU...")
print("  GPU count:", torch.cuda.device_count())
print("  Current device:", torch.cuda.current_device())
print("  Current device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
print()
print("cuDNN properties:")
print("  Version:", torch.backends.cudnn.version())
print("  Enabled:", torch.backends.cudnn.enabled)
print("  Benchmark:", torch.backends.cudnn.benchmark)

In [ ]:
## Define and Check Model Hyperparameters

# modify hyperparameters as needed
hyperparameters.update({
    "epiochs": 10,
    "steps_per_epoch": 2,
    "val_per_epoch": 1,
})

# add path to store training logs and model checkpoints
save_dir:str = '../model_checkpoints/'
os.makedirs(save_dir, exist_ok=True)
hyperparameters['save_dir'] = save_dir

# check hyperparameters
pprint(hyperparameters)

### Checkpoint II
By now you should have:
* Data providers
* A configured CUDA environment
* Custom hyperparameters

## Part III: Construct Data Iterators and Conduct Model Training

In [ ]:
## Build Data Iterators
train_batcher, validation_batcher = BatchProvider.build()

In [ ]:
## Model Setup Test
try:
    train(
        train_loader=train_batcher,
        val_loader=validation_batcher,
        net=assembled_model["model"],
        activation=assembled_model["activation"],
        optimizer=assembled_model["optimizer"],
        loss_fn=assembled_model["loss_fn"],
        device=assembled_model["device"],
        epochs=hyperparameters["epochs"],
        steps_per_epoch=hyperparameters["steps_per_epoch"],
        val_per_epoch=hyperparameters["val_per_epoch"],
        save_dir=hyperparameters["save_dir"],
        write_val_per_feature=True,
        data_range=1,
        uses_11_normalization=False,
        num_channels=2
    )
except Exception as e:
    print("An error occurred during training:")
    print(e)
    pass